In [1]:
%matplotlib inline
import galpy
import numpy as np
import scipy as sp
from matplotlib.pylab import plt, xlim, ylim
from astropy import units as u

from galpy.df import streamdf
from galpy.orbit import Orbit
from galpy.potential import LogarithmicHaloPotential, MWPotential2014, IsochronePotential
from galpy.actionAngle import actionAngleIsochroneApprox
from galpy.util import conversion #for unit conversions

In [2]:
#Parameters
sigv= 9.2 #km/s

#Set up potentials
#lp= LogarithmicHaloPotential(normalize=1.,q=0.9)
mw= MWPotential2014# [2] *1.5
mw[2]*= 1.5

#Set up orbits
#orb_doc= Orbit([1.56148083,0.35081535,-1.15481504,0.88719443,-0.47713334,0.12019596])
#orb_forw= Orbit([337.2675994746023/8, 36.06614670750193/8, 1088.950249792649/8, 
 #               -286.8425553650681/220, 61.58222451126416/220, -66.40339287756248/220]
 #               ,ro=8, vo=220)
#needed to specify ro and vo for orb_forw to convert natural units to physical units

orb_back= Orbit.from_name('Leo I')

#Set up times
back = np.linspace(0,-2,11)*u.Gyr
#forw = np.linspace(0, 1,11)*u.Gyr

In [3]:
#Now we do the same thing but smarter:
#Import function that estimates the correct scale length for an orbit object 
from galpy.actionAngle import estimateBIsochrone

orb_back.integrate(back,mw)
#orb_forw.integrate(forw,mw)

#Finally, estimate the correct scale length to approx the potential over this orbit
#bs_back = estimateBIsochrone(mw,orb_back.R(back),orb_back.z(back))
#bs_forw = estimateBIsochrone(mw,orb_forw.R(forw),orb_forw.z(forw))
bs_back = estimateBIsochrone(mw,orb_back.R(back),orb_back.z(back))
print(f" \n {bs_back}")

 
 [ 6.74562412 21.93727703 30.67529447]


In [ ]:
#With these values, we run our search
b = np.linspace(6.745,30.675,2393)
b_fit = []

for r in b:
    aAI_mw= actionAngleIsochroneApprox(pot=mw, b=r)
    try:
        sdf= streamdf(sigv/220.,progenitor=orb_back, pot=mw, aA=aAI_mw,leading=False,
              nTrackChunks=10, nTrackIterations=2, tdisrupt=2/conversion.time_in_Gyr(220.,8.))
    except np.linalg.LinAlgError:
        print(f"{r} didn't work!")
    except ValueError:
        print(f"{r} produced a value error")
    else:
        print(f"No nans/infs reported. {r} works!")
        b_fit.append(r)

/data/reu/jkittlin/miniconda3/envs/galpyenv/lib/python3.11/site-packages/galpy/actionAngle/actionAngleIsochrone.py:236: RuntimeWarning: invalid value encountered in sqrt
  Jr= self.amp/numpy.sqrt(-2.*E)\

/data/reu/jkittlin/miniconda3/envs/galpyenv/lib/python3.11/site-packages/galpy/actionAngle/actionAngleIsochrone.py:239: RuntimeWarning: invalid value encountered in power
  Omegar= (-2.*E)**1.5/self.amp

/data/reu/jkittlin/miniconda3/envs/galpyenv/lib/python3.11/site-packages/galpy/actionAngle/actionAngleIsochrone.py:263: RuntimeWarning: invalid value encountered in sqrt
  tan11= numpy.arctan(numpy.sqrt((1.+e)/(1.-e))*numpy.tan(0.5*eta))

/data/reu/jkittlin/miniconda3/envs/galpyenv/lib/python3.11/site-packages/galpy/actionAngle/actionAngleIsochrone.py:264: RuntimeWarning: invalid value encountered in sqrt
  tan12= numpy.arctan(numpy.sqrt((1.+e+2.*self.b/c)/(1.-e+2.*self.b/c))*numpy.tan(0.5*eta))

/data/reu/jkittlin/miniconda3/envs/galpyenv/lib/python3.11/site-packages/numpy/linalg/lin

In [6]:
#b_fit = [11.34, 16.29, 21.990000000000002, 39.96, 42.53]
b_fit = bs_back
aAI_mw = actionAngleIsochroneApprox(pot=mw, b=b_fit[1])
sdf = streamdf(sigv/220., progenitor=orb_back, pot=mw, aA=aAI_mw,leading=False,
              nTrackChunks=10, tdisrupt=2/conversion.time_in_Gyr(220.,8.))

/data/reu/jkittlin/miniconda3/envs/galpyenv/lib/python3.11/site-packages/galpy/actionAngle/actionAngleIsochrone.py:236: RuntimeWarning: invalid value encountered in sqrt
  Jr= self.amp/numpy.sqrt(-2.*E)\

/data/reu/jkittlin/miniconda3/envs/galpyenv/lib/python3.11/site-packages/galpy/actionAngle/actionAngleIsochrone.py:239: RuntimeWarning: invalid value encountered in power
  Omegar= (-2.*E)**1.5/self.amp

/data/reu/jkittlin/miniconda3/envs/galpyenv/lib/python3.11/site-packages/galpy/actionAngle/actionAngleIsochrone.py:263: RuntimeWarning: invalid value encountered in sqrt
  tan11= numpy.arctan(numpy.sqrt((1.+e)/(1.-e))*numpy.tan(0.5*eta))

/data/reu/jkittlin/miniconda3/envs/galpyenv/lib/python3.11/site-packages/galpy/actionAngle/actionAngleIsochrone.py:264: RuntimeWarning: invalid value encountered in sqrt
  tan12= numpy.arctan(numpy.sqrt((1.+e+2.*self.b/c)/(1.-e+2.*self.b/c))*numpy.tan(0.5*eta))

/data/reu/jkittlin/miniconda3/envs/galpyenv/lib/python3.11/site-packages/numpy/linalg/lin

LinAlgError: Array must not contain infs or NaNs

In [ ]:
#Plots average location of track
sdf.plotTrack(d1='r',d2='z',interp=True,color='red',spread=2, lw=2.,scaleToPhysical=True, label="Stream")

#Turning off interpolation plots points that were explicitly calculated
sdf.plotTrack(d1='r',d2='z',interp=False,color='yellow',spread=0,overplot=True,ls='none',marker='o',scaleToPhysical=True)

#We also plot the progenitory to compare
sdf.plotProgenitor(d1='r',d2='z',color='k', overplot=True, scaleToPhysical=True, label="Leo I")
plt.legend()

plt.xlim(10,300)
plt.ylim(-200,500)
"""Bro these guys are not going in the same direction"""

In [ ]:
sdf.plotTrack(d1='ll',d2='dist',interp=True,color='r',spread=2,overplot=False,lw=2., label="Stream")
sdf.plotTrack(d1='ll',d2='dist',interp=False,color='y',spread=0,overplot=True,ls='none',marker='o')
sdf.plotProgenitor(d1='ll',d2='dist',color='k',overplot=True,ls='--', label="Leo I")
xlim(0,360); ylim(10,500)
